In [1]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Activation
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
import os
import cv2

from imutils import paths
import matplotlib.pyplot as plt
from keras.preprocessing import image

import argparse
import imutils

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                 shear_range = 0.2,
                 zoom_range = 0.2,
                 horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)


training_set = train_datagen.flow_from_directory('/content/drive/My Drive/augmented/image_data/train',
                target_size = (128, 128),
                batch_size = 32,
                class_mode = 'categorical')
test_set= test_datagen.flow_from_directory('/content/drive/My Drive/augmented/image_data/validation',
                  target_size = (128, 128),
                  batch_size = 32,
                  class_mode = 'categorical')


Found 2121 images belonging to 3 classes.
Found 63 images belonging to 3 classes.


In [5]:
train_dir='/content/drive/My Drive/augmented/image_data/train'
test_dir='/content/drive/My Drive/augmented/image_data/validation'



train_dir_saree=train_dir+ '/saree'
train_dir_saree=train_dir + '/saree'
train_dir_men_shrts = train_dir + '/shirt'
test_dir_men_shrts = test_dir + '/shirt'
train_dir_men_tshrts = train_dir + '/tshirt'
test_dir_men_tshrts = test_dir + '/tshirt'

In [6]:
print('number of shirts training images - ',len(os.listdir(train_dir_men_shrts)))
print('number of tshirts training images - ',len(os.listdir(train_dir_men_tshrts)))
print('number of shirts testing images - ',len(os.listdir(test_dir_men_shrts)))
print('number of tshirts testing images - ',len(os.listdir(test_dir_men_tshrts)))

number of shirts training images -  707
number of tshirts training images -  707
number of shirts testing images -  21
number of tshirts testing images -  21


In [76]:
model = Sequential()

model.add(Conv2D(32,(3,3), padding = 'same',
                input_shape=(128, 128, 3)))
model.add(Activation('relu'))
model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64,(3,3),padding = 'same'))
model.add(Activation('relu'))
model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(3))
model.add(Activation('softmax'))
#initiate RMSProp optimizer and configure parameters
opt = keras.optimizers.rmsprop(lr=0.0001,decay = 1e-6)
#let's create our model
model.compile(loss = 'categorical_crossentropy',optimizer = opt, metrics = ['accuracy'])

print(model.summary())
model.fit_generator(training_set,epochs=20,validation_data=test_set)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
activation_13 (Activation)   (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 126, 126, 32)      9248      
_________________________________________________________________
activation_14 (Activation)   (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 63, 63, 64)        18496     
_________________________________________________________________
activation_15 (Activation)   (None, 63, 63, 64)       

In [ ]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = 25
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Image")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")


In [ ]:
model.save('image_classification')
print('model saved')

In [ ]:
from keras.models import load_model
classifier = load_model('image_classification')